<a href="https://colab.research.google.com/github/Chrisini/DecentNet/blob/master/weak_vessel_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 𝕎𝕖𝕒𝕜 𝕧𝕖𝕤𝕤𝕖𝕝 𝕞𝕒𝕤𝕜 𝕘𝕖𝕟𝕖𝕣𝕒𝕥𝕚𝕠𝕟

* In order to understand our concept clusters and neural network results, we want to get vessel information.

# Import

In [1]:
import os
import glob

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset
# import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, ToPILImage, CenterCrop

if True:
    pass
  #pip install bunch
  #from bunch import Bunch

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

if False:
    train_path =     r"/content/drive/MyDrive/decentnet/data/images/train/AMD"
elif False:
    train_path =     r"/content/drive/MyDrive/decentnet/data/images/train/nonAMD"
elif True:
    train_path =     r"C:/Users/Prinzessin/projects/image_data/RAVIR_Dataset/train/training_images"
    #train_path = r"C:\snec_data\Result_Data\cirrus\enface"
    #train_path = r"C:\snec_data\Result_Data\cirrus\superenface"

ckpt_vessel_path =   r"example_ckpt/vessel_unet_w_liu"

vessel_path =        r"example_results/Vessel_Masks" # results
vessel_path_thresh = r"example_results/Vessel_Masks_Thresholded" # results


# FR Unet

This code is stolen and adapted from: https://github.com/lseventeen/FR-UNet
Liu, Wentao, et al. "Full-resolution network and dual-threshold iteration for retinal vessel and coronary angiograph segmentation." IEEE Journal of Biomedical and Health Informatics 26.9 (2022): 4623-4634.

In [3]:
class vessel_dataset(Dataset):
    def __init__(self, path):

        self.data_path = path
        self.data_file = os.listdir(self.data_path)
        
        self.transforms = Compose([
            # Resize((592, 592)), # 592
            Resize((256, 256)),
            CenterCrop((96, 96)),
            ToTensor(),
            # torchvision.transforms.RandomInvert(p=1),
            # Normalize((0.7, ), (0.7, )) # grayscale
        ])

    def __getitem__(self, idx):
        path = os.path.join(self.data_path, self.data_file[idx])
        
        img = Image.open(path).convert("L")
        if True:
            mask = Image.open(path.replace("training_images", "training_masks")).convert("L")
        #red, img, blue = img.split()
        size = img.size
        img = self.transforms(img)
        mask = self.transforms(mask)
        return img, mask, self.data_file[idx], size

    def __len__(self):
        return len(self.data_file)

In [4]:
592 * 0.75

444.0

In [5]:
class conv(nn.Module):
    def __init__(self, in_c, out_c, dp=0):
        super(conv, self).__init__()
        self.in_c = in_c
        self.out_c = out_c
        self.conv = nn.Sequential(
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.Dropout2d(dp),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.Dropout2d(dp),
            nn.LeakyReLU(0.1, inplace=True))
        self.relu = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        res = x
        x = self.conv(x)
        out = x + res
        out = self.relu(out)
        return x


class feature_fuse(nn.Module):
    def __init__(self, in_c, out_c):
        super(feature_fuse, self).__init__()
        self.conv11 = nn.Conv2d(
            in_c, out_c, kernel_size=1, padding=0, bias=False)
        self.conv33 = nn.Conv2d(
            in_c, out_c, kernel_size=3, padding=1, bias=False)
        self.conv33_di = nn.Conv2d(
            in_c, out_c, kernel_size=3, padding=2, bias=False, dilation=2)
        self.norm = nn.BatchNorm2d(out_c)

    def forward(self, x):
        x1 = self.conv11(x)
        x2 = self.conv33(x)
        x3 = self.conv33_di(x)
        out = self.norm(x1+x2+x3)
        return out


class up(nn.Module):
    def __init__(self, in_c, out_c, dp=0):
        super(up, self).__init__()
        self.up = nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, kernel_size=2,
                               padding=0, stride=2, bias=False),
            nn.BatchNorm2d(out_c),
            nn.LeakyReLU(0.1, inplace=False))

    def forward(self, x):
        x = self.up(x)
        return x


class down(nn.Module):
    def __init__(self, in_c, out_c, dp=0):
        super(down, self).__init__()
        self.down = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=2,
                      padding=0, stride=2, bias=False),
            nn.BatchNorm2d(out_c),
            nn.LeakyReLU(0.1, inplace=True))

    def forward(self, x):
        x = self.down(x)
        return x


class block(nn.Module):
    def __init__(self, in_c, out_c,  dp=0, is_up=False, is_down=False, fuse=False):
        super(block, self).__init__()
        self.in_c = in_c
        self.out_c = out_c
        if fuse == True:
            self.fuse = feature_fuse(in_c, out_c)
        else:
            self.fuse = nn.Conv2d(in_c, out_c, kernel_size=1, stride=1)

        self.is_up = is_up
        self.is_down = is_down
        self.conv = conv(out_c, out_c, dp=dp)
        if self.is_up == True:
            self.up = up(out_c, out_c//2)
        if self.is_down == True:
            self.down = down(out_c, out_c*2)

    def forward(self,  x):
        if self.in_c != self.out_c:
            x = self.fuse(x)
        x = self.conv(x)
        if self.is_up == False and self.is_down == False:
            return x
        elif self.is_up == True and self.is_down == False:
            x_up = self.up(x)
            return x, x_up
        elif self.is_up == False and self.is_down == True:
            x_down = self.down(x)
            return x, x_down
        else:
            x_up = self.up(x)
            x_down = self.down(x)
            return x, x_up, x_down


class FR_UNet(nn.Module):
    def __init__(self,  num_classes=1, num_channels=1, feature_scale=2,  dropout=0.2, fuse=True, out_ave=True):
        super(FR_UNet, self).__init__()
        self.out_ave = out_ave
        filters = [64, 128, 256, 512, 1024]
        filters = [int(x / feature_scale) for x in filters]
        self.block1_3 = block(
            num_channels, filters[0],  dp=dropout, is_up=False, is_down=True, fuse=fuse)
        self.block1_2 = block(
            filters[0], filters[0],  dp=dropout, is_up=False, is_down=True, fuse=fuse)
        self.block1_1 = block(
            filters[0]*2, filters[0],  dp=dropout, is_up=False, is_down=True, fuse=fuse)
        self.block10 = block(
            filters[0]*2, filters[0],  dp=dropout, is_up=False, is_down=True, fuse=fuse)
        self.block11 = block(
            filters[0]*2, filters[0],  dp=dropout, is_up=False, is_down=True, fuse=fuse)
        self.block12 = block(
            filters[0]*2, filters[0],  dp=dropout, is_up=False, is_down=False, fuse=fuse)
        self.block13 = block(
            filters[0]*2, filters[0],  dp=dropout, is_up=False, is_down=False, fuse=fuse)
        self.block2_2 = block(
            filters[1], filters[1],  dp=dropout, is_up=True, is_down=True, fuse=fuse)
        self.block2_1 = block(
            filters[1]*2, filters[1],  dp=dropout, is_up=True, is_down=True, fuse=fuse)
        self.block20 = block(
            filters[1]*3, filters[1],  dp=dropout, is_up=True, is_down=True, fuse=fuse)
        self.block21 = block(
            filters[1]*3, filters[1],  dp=dropout, is_up=True, is_down=False, fuse=fuse)
        self.block22 = block(
            filters[1]*3, filters[1],  dp=dropout, is_up=True, is_down=False, fuse=fuse)
        self.block3_1 = block(
            filters[2], filters[2],  dp=dropout, is_up=True, is_down=True, fuse=fuse)
        self.block30 = block(
            filters[2]*2, filters[2],  dp=dropout, is_up=True, is_down=False, fuse=fuse)
        self.block31 = block(
            filters[2]*3, filters[2],  dp=dropout, is_up=True, is_down=False, fuse=fuse)
        self.block40 = block(filters[3], filters[3],
                             dp=dropout, is_up=True, is_down=False, fuse=fuse)
        self.final1 = nn.Conv2d(
            filters[0], num_classes, kernel_size=1, padding=0, bias=True)
        self.final2 = nn.Conv2d(
            filters[0], num_classes, kernel_size=1, padding=0, bias=True)
        self.final3 = nn.Conv2d(
            filters[0], num_classes, kernel_size=1, padding=0, bias=True)
        self.final4 = nn.Conv2d(
            filters[0], num_classes, kernel_size=1, padding=0, bias=True)
        self.final5 = nn.Conv2d(
            filters[0], num_classes, kernel_size=1, padding=0, bias=True)
        self.fuse = nn.Conv2d(
            5, num_classes, kernel_size=1, padding=0, bias=True)
        # self.apply(InitWeights_He)

    def forward(self, x):
        x1_3, x_down1_3 = self.block1_3(x)
        x1_2, x_down1_2 = self.block1_2(x1_3)
        x2_2, x_up2_2, x_down2_2 = self.block2_2(x_down1_3)
        x1_1, x_down1_1 = self.block1_1(torch.cat([x1_2, x_up2_2], dim=1))
        x2_1, x_up2_1, x_down2_1 = self.block2_1(
            torch.cat([x_down1_2, x2_2], dim=1))
        x3_1, x_up3_1, x_down3_1 = self.block3_1(x_down2_2)
        x10, x_down10 = self.block10(torch.cat([x1_1, x_up2_1], dim=1))
        x20, x_up20, x_down20 = self.block20(
            torch.cat([x_down1_1, x2_1, x_up3_1], dim=1))
        x30, x_up30 = self.block30(torch.cat([x_down2_1, x3_1], dim=1))
        _, x_up40 = self.block40(x_down3_1)
        x11, x_down11 = self.block11(torch.cat([x10, x_up20], dim=1))
        x21, x_up21 = self.block21(torch.cat([x_down10, x20, x_up30], dim=1))
        _, x_up31 = self.block31(torch.cat([x_down20, x30, x_up40], dim=1))
        x12 = self.block12(torch.cat([x11, x_up21], dim=1))
        _, x_up22 = self.block22(torch.cat([x_down11, x21, x_up31], dim=1))
        x13 = self.block13(torch.cat([x12, x_up22], dim=1))
        if self.out_ave == True:
            output = (self.final1(x1_1)+self.final2(x10) +
                      self.final3(x11)+self.final4(x12)+self.final5(x13))/5
        else:
            output = self.final5(x13)




        return output

# Trainer

In [6]:
import os
import pickle
import random
import cv2
import numpy as np
import torch
from torchvision.transforms import functional as F

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


class Fix_RandomRotation(object):

    def __init__(self, degrees=360, resample=False, expand=False, center=None):
        self.degrees = degrees
        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params():
        p = torch.rand(1)

        if p >= 0 and p < 0.25:
            angle = -180
        elif p >= 0.25 and p < 0.5:
            angle = -90
        elif p >= 0.5 and p < 0.75:
            angle = 90
        else:
            angle = 0
        return angle

    def __call__(self, img):
        angle = self.get_params()
        return F.rotate(img, angle, self.resample, self.expand, self.center)

    def __repr__(self):
        format_string = self.__class__.__name__ + \
            '(degrees={0}'.format(self.degrees)
        format_string += ', resample={0}'.format(self.resample)
        format_string += ', expand={0}'.format(self.expand)
        if self.center is not None:
            format_string += ', center={0}'.format(self.center)
        format_string += ')'
        return format_string


def dir_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)


def remove_files(path):
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))


def read_pickle(path, type):
    with open(file=path + f"/{type}.pkl", mode='rb') as file:
        img = pickle.load(file)
    return img


def save_pickle(path, type, img_list):
    with open(file=path + f"/{type}.pkl", mode='wb') as file:
        pickle.dump(img_list, file)


def double_threshold_iteration(index,img, h_thresh, l_thresh, save=True):
    h, w = img.shape
    img = np.array(torch.sigmoid(img).cpu().detach()*255, dtype=np.uint8)
    bin = np.where(img >= h_thresh*255, 255, 0).astype(np.uint8)
    gbin = bin.copy()
    gbin_pre = gbin-1
    while(gbin_pre.all() != gbin.all()):
        gbin_pre = gbin
        for i in range(h):
            for j in range(w):
                if gbin[i][j] == 0 and img[i][j] < h_thresh*255 and img[i][j] >= l_thresh*255:
                    if gbin[i-1][j-1] or gbin[i-1][j] or gbin[i-1][j+1] or gbin[i][j-1] or gbin[i][j+1] or gbin[i+1][j-1] or gbin[i+1][j] or gbin[i+1][j+1]:
                        gbin[i][j] = 255

    if save:
        cv2.imwrite(f"save_picture/bin{index}.png", bin)
        cv2.imwrite(f"save_picture/gbin{index}.png", gbin)
    return gbin/255


def recompone_overlap(preds, img_h, img_w, stride_h, stride_w):
    assert (len(preds.shape) == 4)
    assert (preds.shape[1] == 1 or preds.shape[1] == 3)
    patch_h = preds.shape[2]
    patch_w = preds.shape[3]
    N_patches_h = (img_h - patch_h) // stride_h + 1
    N_patches_w = (img_w - patch_w) // stride_w + 1
    N_patches_img = N_patches_h * N_patches_w
    assert (preds.shape[0] % N_patches_img == 0)
    N_full_imgs = preds.shape[0] // N_patches_img
    full_prob = np.zeros((N_full_imgs, preds.shape[1], img_h, img_w))
    full_sum = np.zeros((N_full_imgs, preds.shape[1], img_h, img_w))
    k = 0
    for i in range(N_full_imgs):
        for h in range((img_h - patch_h) // stride_h + 1):
            for w in range((img_w - patch_w) // stride_w + 1):
                full_prob[i, :, h * stride_h:(h * stride_h) + patch_h, w * stride_w:(w * stride_w) + patch_w] += preds[
                    k]
                full_sum[i, :, h * stride_h:(h * stride_h) + patch_h,
                         w * stride_w:(w * stride_w) + patch_w] += 1
                k += 1
    assert (k == preds.shape[0])
    assert (np.min(full_sum) >= 1.0)
    final_avg = full_prob / full_sum
    return final_avg

import numpy as np
import torch
import cv2
from sklearn.metrics import roc_auc_score


class AverageMeter(object):
    def __init__(self):
        self.initialized = False
        self.val = None
        self.avg = None
        self.sum = None
        self.count = None

    def initialize(self, val, weight):
        self.val = val
        self.avg = val
        self.sum = np.multiply(val, weight)
        self.count = weight
        self.initialized = True

    def update(self, val, weight=1):
        if not self.initialized:
            self.initialize(val, weight)
        else:
            self.add(val, weight)

    def add(self, val, weight):
        self.val = val
        self.sum = np.add(self.sum, np.multiply(val, weight))
        self.count = self.count + weight
        self.avg = self.sum / self.count

    @property
    def value(self):
        return np.round(self.val, 4)

    @property
    def average(self):
        return np.round(self.avg, 4)


def get_metrics(predict, target, threshold=None, predict_b=None):
    predict = torch.sigmoid(predict).cpu().detach().numpy().flatten()
    if predict_b is not None:
        predict_b = predict_b.flatten()
    else:
        predict_b = np.where(predict >= threshold, 1, 0)
    if torch.is_tensor(target):
        target = target.cpu().detach().numpy().flatten()
    else:
        target = target.flatten()
    tp = (predict_b * target).sum()
    tn = ((1 - predict_b) * (1 - target)).sum()
    fp = ((1 - target) * predict_b).sum()
    fn = ((1 - predict_b) * target).sum()
    auc = roc_auc_score(target, predict)
    acc = (tp + tn) / (tp + fp + fn + tn)
    pre = tp / (tp + fp)
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    iou = tp / (tp + fp + fn)
    f1 = 2 * pre * sen / (pre + sen)
    return {
        "AUC": np.round(auc, 4),
        "F1": np.round(f1, 4),
        "Acc": np.round(acc, 4),
        "Sen": np.round(sen, 4),
        "Spe": np.round(spe, 4),
        "pre": np.round(pre, 4),
        "IOU": np.round(iou, 4),
    }


def count_connect_component(predict, target, threshold=None, connectivity=8):
    if threshold != None:
        predict = torch.sigmoid(predict).cpu().detach().numpy()
        predict = np.where(predict >= threshold, 1, 0)
    if torch.is_tensor(target):
        target = target.cpu().detach().numpy()
    pre_n, _, _, _ = cv2.connectedComponentsWithStats(np.asarray(
        predict, dtype=np.uint8)*255, connectivity=connectivity)
    gt_n, _, _, _ = cv2.connectedComponentsWithStats(np.asarray(
        target, dtype=np.uint8)*255, connectivity=connectivity)
    return pre_n/gt_n

In [7]:
import torch.nn


class DiceLoss(torch.nn.Module):
    # Source: https://github.com/HiLab-git/SSL4MIS/blob/master/code/utils/losses.py
    def __init__(self, n_output_neurons, softmax=False):
        super(DiceLoss, self).__init__()
        self.n_output_neurons = n_output_neurons
        self.softmax=softmax

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_output_neurons):
            temp_prob = input_tensor == i * torch.ones_like(input_tensor)
            tensor_list.append(temp_prob)
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _dice_loss(self, model_output, ground_truth):
        ground_truth = ground_truth.float()
        smooth = 1e-5
        intersect = torch.sum(model_output * ground_truth)
        y_sum = torch.sum(ground_truth * ground_truth)
        z_sum = torch.sum(model_output * model_output)
        loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
        loss = 1 - loss
        return loss

    def forward(self, model_output, ground_truth, weight=None):
        if self.softmax:
            model_output = torch.softmax(model_output, dim=1)
        ground_truth = self._one_hot_encoder(ground_truth)
        
        if weight is None:
            weight = [1] * self.n_output_neurons
        assert model_output.size() == ground_truth.size(), 'predict & ground_truth shape do not match'
        class_wise_dice = []
        loss = 0.0
        for i in range(0, self.n_output_neurons):
            dice = self._dice_loss(model_output[:, i], ground_truth[:, i])
            class_wise_dice.append(1.0 - dice.item())
            loss += dice * weight[i]
        return loss / self.n_output_neurons

In [8]:
import os
import time
from datetime import datetime
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.transforms.functional as TF
from loguru import logger
from torch.utils import tensorboard
from tqdm import tqdm
# from utils.helpers import dir_exists, get_instance, remove_files, double_threshold_iteration
# from utils.metrics import AverageMeter, get_metrics, get_metrics, count_connect_component
import ttach as tta


class Trainer:
    def __init__(self, model, loss=None, train_loader=None, val_loader=None):

        self.epochs= 40
        self.save_dir= "example_results/"
        self.val_per_epochs= 1
        self.save_period= 1
        # self.tensorboard= true
        self.threshold= 0.5
        self.threshold_low= 0.3
        #self.DTI= false
        #self.amp= true
        #self.tta= false
        #self.CCC= false
        
        self.loss = DiceLoss(3)
        self.model = nn.DataParallel(model.cuda())
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optim = torch.optim.Adam(self.model.parameters(), lr=0.001, weight_decay=0.00001) # get_instance(torch.optim, "optimizer", CFG, self.model.parameters())
        self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optim, 
                                                                              T_0 = 5, # number of iterations for the first restart.
                                                                              eta_min = 0.00001
                                                                               ) # get_instance(torch.optim.lr_scheduler, "lr_scheduler", CFG, self.optimizer)


        start_time = datetime.now().strftime('%y%m%d%H%M%S')
        self.checkpoint_dir = os.path.join(self.save_dir, start_time)
        self.writer = tensorboard.SummaryWriter(self.checkpoint_dir)
        dir_exists(self.checkpoint_dir)
        cudnn.benchmark = True
        
        
        

    def train(self):
        for epoch in range(1, self.epochs + 1):
            self._train_epoch(epoch)
            if self.val_loader is not None and epoch % self.val_per_epochs == 0:
                results = self._valid_epoch(epoch)
                logger.info(f'## Info for epoch {epoch} ## ')
                for k, v in results.items():
                    logger.info(f'{str(k):15s}: {v}')
            if epoch % 10 == 0:
                self._save_checkpoint(epoch)

    def _train_epoch(self, epoch):
        self.model.train()
        wrt_mode = 'train'
        self._reset_metrics()
        tbar = tqdm(self.train_loader, ncols=160)
        tic = time.time()
        for img, gt, _, _ in tbar:
            self.data_time.update(time.time() - tic)
            img = img.cuda(non_blocking=True)
            gt = gt.cuda(non_blocking=True)
            self.optim.zero_grad()
            
            pre = self.model(img)
            
            #print(pre.shape)
            #print(gt.shape)
            
            loss = self.loss(pre, gt)
            loss.backward()
            self.optim.step()
            
            self.total_loss.update(loss.item())
            self.batch_time.update(time.time() - tic)
            
            """
            self._metrics_update(
                *get_metrics(pre, gt, threshold=self.threshold).values())
            tbar.set_description(
                'TRAIN ({}) | Loss: {:.4f} | AUC {:.4f} F1 {:.4f} Acc {:.4f}  Sen {:.4f} Spe {:.4f} Pre {:.4f} IOU {:.4f} |B {:.2f} D {:.2f} |'.format(
                    epoch, self.total_loss.average, *self._metrics_ave().values(), self.batch_time.average, self.data_time.average))
            """
            tic = time.time()
        self.writer.add_scalar(
            f'{wrt_mode}/loss', self.total_loss.average, epoch)
        """
        for k, v in list(self._metrics_ave().items())[:-1]:
            self.writer.add_scalar(f'{wrt_mode}/{k}', v, epoch)
        """
        for i, opt_group in enumerate(self.optim.param_groups):
            self.writer.add_scalar(
                f'{wrt_mode}/Learning_rate_{i}', opt_group['lr'], epoch)
        self.lr_scheduler.step()

    def _valid_epoch(self, epoch):
        logger.info('\n###### EVALUATION ######')
        self.model.eval()
        wrt_mode = 'val'
        """
        self._reset_metrics()
        """
        tbar = tqdm(self.val_loader, ncols=160)
        with torch.no_grad():
            for img, gt, _, _ in tbar:
                img = img.cuda(non_blocking=True)
                gt = gt.cuda(non_blocking=True)
                
                predict = self.model(img)
                loss = self.loss(predict, gt)
                self.total_loss.update(loss.item())
                """
                self._metrics_update(
                    *get_metrics(predict, gt, threshold=self.threshold).values())
                tbar.set_description(
                    'EVAL ({})  | Loss: {:.4f} | AUC {:.4f} F1 {:.4f} Acc {:.4f} Sen {:.4f} Spe {:.4f} Pre {:.4f} IOU {:.4f} |'.format(
                        epoch, self.total_loss.average, *self._metrics_ave().values()))
                """
                self.writer.add_scalar(
                    f'{wrt_mode}/loss', self.total_loss.average, epoch)

        self.writer.add_scalar(
            f'{wrt_mode}/loss', self.total_loss.average, epoch)
        """
        for k, v in list(self._metrics_ave().items())[:-1]:
            self.writer.add_scalar(f'{wrt_mode}/{k}', v, epoch)
        """
        log = {
            'val_loss': self.total_loss.average,
            #**self._metrics_ave()
        }
        
        return log

    def _save_checkpoint(self, epoch):
        state = {
            'arch': type(self.model).__name__,
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'optim': self.optim.state_dict()
        }
        filename = os.path.join(self.checkpoint_dir,
                                f'checkpoint-epoch{epoch}.pth')
        logger.info(f'Saving a checkpoint: {filename} ...')
        torch.save(state, filename)
        return filename

    def _reset_metrics(self):
        self.batch_time = AverageMeter()
        self.data_time = AverageMeter()
        self.total_loss = AverageMeter()
        self.auc = AverageMeter()
        self.f1 = AverageMeter()
        self.acc = AverageMeter()
        self.sen = AverageMeter()
        self.spe = AverageMeter()
        self.pre = AverageMeter()
        self.iou = AverageMeter()
        self.CCC = AverageMeter()
    def _metrics_update(self, auc, f1, acc, sen, spe, pre, iou):
        self.auc.update(auc)
        self.f1.update(f1)
        self.acc.update(acc)
        self.sen.update(sen)
        self.spe.update(spe)
        self.pre.update(pre)
        self.iou.update(iou)

    def _metrics_ave(self):

        return {
            "AUC": self.auc.average,
            "F1": self.f1.average,
            "Acc": self.acc.average,
            "Sen": self.sen.average,
            "Spe": self.spe.average,
            "pre": self.pre.average,
            "IOU": self.iou.average
        }

# Tester

In [20]:
import PIL.ImageOps   
class Tester():
    def __init__(self, model, checkpoint, test_loader, dataset_path):

        # test loader has to have batch size = 1

        self.test_loader = test_loader
        self.model = model

    def test(self):
        
        self.model.eval()
        with torch.no_grad():
            for _, (img, _, image_path, size) in enumerate(self.test_loader):

                cur_img_name = os.path.splitext(os.path.basename(os.path.normpath(image_path[0])))[0]

                pred = self.model(img)

                # predict = torch.sigmoid(pred).cpu().detach().numpy()
                
                # predict = pred.cpu().detach().numpy()
                
                #print(pred.shape)
                
                pred, classes = torch.max(pred, dim=1)
                
                
                predict = classes.cpu().detach().numpy()
                
                #print(predict.shape)
                
                # predict = predict.transpose(2,3,1,0).squeeze()
                predict = predict.transpose(1, 2, 0).squeeze()
                
                #print(predict.shape)
                
                print(np.unique(predict))
                
                im1 = Image.fromarray(np.uint8(predict*100))
                
                #print(im1)
                
                # im1 = PIL.ImageOps.invert(im1)
                im1 = im1.resize(size)
                im1 = im1.save(os.path.join( vessel_path, f"{cur_img_name}.png" )) # _pre_orig_{i}

                """
                predict_b = np.where(predict >= 0.15, 1, 0) # first value is the threshold
                pb = predict_b.transpose(2,3,1,0).squeeze()
                im2 = Image.fromarray(np.uint8(pb*255))
                # im2 = PIL.ImageOps.invert(im2)
                im2 = im2.resize(size)
                im2 = im2.save(os.path.join( vessel_path_thresh, f"{cur_img_name}.png" ))
                """

# Main

In [21]:
if False:
    data_path = train_path
    weight_path = os.path.join(ckpt_vessel_path, "drive-epoch40.pth")

    result_path = vessel_path

    checkpoint = torch.load(weight_path, map_location=torch.device("cpu"))

    test_dataset = vessel_dataset(data_path)
    test_loader = DataLoader(test_dataset, 1, shuffle=False,  num_workers=0, pin_memory=True)

    model = FR_UNet(num_classes=1, num_channels=1, feature_scale=2,  dropout=0.2, fuse=True, out_ave=True)
    for key in list(checkpoint['state_dict'].keys()):
                checkpoint['state_dict'][key.replace('module.', '')] = checkpoint['state_dict'].pop(key)
    model.load_state_dict(checkpoint['state_dict'])


    filters = [int(64/2)]
    num_classes = 3 # bg, vessel1, vessel2
    model.final1 = nn.Conv2d(
        filters[0], num_classes, kernel_size=1, padding=0, bias=True)
    model.final2 = nn.Conv2d(
        filters[0], num_classes, kernel_size=1, padding=0, bias=True)
    model.final3 = nn.Conv2d(
        filters[0], num_classes, kernel_size=1, padding=0, bias=True)
    model.final4 = nn.Conv2d(
        filters[0], num_classes, kernel_size=1, padding=0, bias=True)
    model.final5 = nn.Conv2d(
        filters[0], num_classes, kernel_size=1, padding=0, bias=True)
    model.fuse = nn.Conv2d(
        5, num_classes, kernel_size=1, padding=0, bias=True)


    train = Trainer(model, train_loader=test_loader, val_loader=test_loader)
    train.train()







In [22]:

data_path = train_path
weight_path = r"C:\Users\Prinzessin\projects\decentnet\datasceyence\examples\example_results\230713102104\checkpoint-epoch40.pth"
result_path = vessel_path

checkpoint = torch.load(weight_path, map_location=torch.device("cpu"))

test_dataset = vessel_dataset(data_path)
test_loader = DataLoader(test_dataset, 1, shuffle=False,  num_workers=0, pin_memory=True)

model = FR_UNet(num_classes=3, num_channels=1, feature_scale=2,  dropout=0.2, fuse=True, out_ave=True)
for key in list(checkpoint['state_dict'].keys()):
            checkpoint['state_dict'][key.replace('module.', '')] = checkpoint['state_dict'].pop(key)
model.load_state_dict(checkpoint['state_dict'])

test = Tester(model, checkpoint, test_loader, data_path)
test.test()


[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1]


In [ ]:
18432/2

In [ ]:
data_file = os.listdir(train_path)[0]
path = os.path.join(train_path, data_file)

img = Image.open(path) # .convert("L")

# red, green, blue = img.split()

t = ToTensor()
a = Normalize((0.5, ), (0.5, ))
p = ToPILImage()
 
plt.figure()
plt.imshow(p((t(img))), cmap="gray")
plt.figure()
plt.imshow(p((t(img))), cmap="gray")
plt.figure()
plt.imshow(p((t(img))), cmap="gray")
